# Setting up in the new network / for new accounts:
- This notebook allows to prepare wallets, tokens and contracts for running scenarios.
- You would need to update the consts in the [/scripts/defaults.py](../../scripts/defaults.py) to use deployed tokens and contracts in the scenarios.

In [1]:
from docs.scenarios.setup import *

web3, etherlink_account, tezos_account = setup()

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `100.373892 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `1.65279776 ꜩ`


### Deposit XTZ to L2:
- This call will fund default etherlink_account on Etherlink side by utilizing native deposit bridge.

In [2]:
opg_hash = xtz_deposit.callback(
    xtz_ticket_helper=XTZ_TICKET_HELPER,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    amount=1_000_000,
    receiver_address=etherlink_account.address,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
)

Making XTZ deposit using Helper `KT1MJxf4KVN3sosR99VRG7WBbWTJtAyWUJt9`:
  - Executor: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/parisnet/`
  - XTZ deposit params:
      * Smart Rollup address: `sr1G9N4WzRF79GeGKVd1vUDUg2NSrFGNGAfA`
      * Receiver address: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
      * Amount (mutez): `1_000_000`
Successfully executed XTZ deposit, tx hash: `ooDUcgKbevR33UV23MgrDNfGK71Q7ZN8pTczwHgaHXVRX7VsDKa`


### Deploy L1 token:
- This call will deploy new token, that can be used as a testing token in the scenarios.
- This token should be stored in the Ticketer contract storage to be used in other test scenarios.

In [3]:
token = deploy_token.callback(
    token_type='FA2',
    token_id=9000,
    total_supply=70_000_000,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    skip_confirm=True,
)

- You will need to update the `TEZOS_TOKEN_ADDRESS` and `TEZOS_TOKEN_TYPE` variables in the [/scripts/defaults.py](../../scripts/defaults.py), to use this token contract in the test scenarios.

In [4]:
click.echo("TEZOS_TOKEN_ADDRESS = '" + accent(token.address) + "'")
standard = token.make_token_info()['token_type']
click.echo("TEZOS_TOKEN_TYPE = '" + accent(standard) + "'")

TEZOS_TOKEN_ADDRESS = 'KT1Wsem3Uka7wx5cGnk4nRZfyWVX5J7aWVX4'
TEZOS_TOKEN_TYPE = 'FA2'


### Deploy bridge:
- This call will deploy bridge contracts on both Tezos and Etherlink side for the provided token.
- You can both use existing token from the network or deploy you own test token.

In [5]:
tezos_bridge_contracts = bridge_token.callback(
    token_address=token.address,
    token_type='FA2',
    token_id=9000,
    token_decimals=0,
    token_symbol='vSecT',
    token_name='Various Security Testing Scenarios',
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
    kernel_address=KERNEL_ADDRESS,
    skip_confirm=True
)

ticketer = tezos_bridge_contracts['ticketer']
erc20 = tezos_bridge_contracts['erc20']
token_bridge_helper = tezos_bridge_contracts['token_bridge_helper']

Deploying bridge contracts for `vSecT`:
  - Token contract: `KT1Wsem3Uka7wx5cGnk4nRZfyWVX5J7aWVX4`
  - Token id: `9000`

Deploying Ticketer for `vSecT`:
  - Deployer: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/parisnet/`
  - Params:
      * Token type: `FA2`
      * Token address: `KT1Wsem3Uka7wx5cGnk4nRZfyWVX5J7aWVX4`
      * Token id: `9000`
      * Token symbol: `vSecT`
      * Token name: `Various Security Testing Scenarios`
      * Token decimals: `0`
Successfully deployed Ticketer, address: `KT1Cm6eSN3aEPPuTEQEXkehYBTu6caG5ZNxp`

Deploying ERC20 Proxy for `vSecT`:
  - Deployer: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://etherlink.dipdup.net`
  - Constructor params:
      * Ticketer address bytes: `0x012dd49724d1aad5dea5a1e3c24ffe53322f03957400`
      * Content bytes: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b54315773656d33556b613777783563476e6b346e525a66795

- You will need to update the `TICKETER_ADDRESS`, `ERC20_PROXY_ADDRESS`, `TOKEN_BRIDGE_HELPER_ADDRESS` variables in the [/scripts/defaults.py](../../scripts/defaults.py), to use these contracts in other scenarios.

In [6]:
click.echo("TICKETER_ADDRESS = '" + accent(ticketer.address) + "'")
click.echo("ERC20_PROXY_ADDRESS = '" + accent(erc20.address) + "'")
click.echo("TOKEN_BRIDGE_HELPER_ADDRESS = '" + accent(token_bridge_helper.address) + "'")

TICKETER_ADDRESS = 'KT1Cm6eSN3aEPPuTEQEXkehYBTu6caG5ZNxp'
ERC20_PROXY_ADDRESS = '0x8Cd786B61D1245cbAc2344152a6E35e5e0B5ac8F'
TOKEN_BRIDGE_HELPER_ADDRESS = 'KT1RTYQ1p8fRFpof7EsD1J6stv1oDP4gGDUU'


### Deploy TicketRouterTester:
- This is the tester-contract allowing to make arbitrary calls to the rollup, ticketer, accept tickets with different entrypoints, redirect them, mint tickets.

In [8]:
ticket_router_tester = deploy_ticket_router_tester(
    tezos_account,
    TEZOS_RPC_URL,
)

Deploying TicketRouterTester:
  - Deployer: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/parisnet/`
Successfully deployed TicketRouterTester, address: `KT1PEDAdZEW9SebUUd94VQsRTnJsbkmmmAnM`


- You will need to update the `TICKET_ROUTER_TESTER_ADDRESS` variable in the [/scripts/defaults.py](../../scripts/defaults.py), to use this contracts in other scenarios.

In [9]:
click.echo("TICKET_ROUTER_TESTER_ADDRESS = '" + accent(ticket_router_tester.address) + "'")

TICKET_ROUTER_TESTER_ADDRESS = 'KT1PEDAdZEW9SebUUd94VQsRTnJsbkmmmAnM'


### Wrap tokens to tickets:
- To use tickets, that represent FA token, directly from tezos account, they should be converted without sending to the rollup.
- There is a helper to do this:

In [10]:
opg_hash = wrap_tokens_to_tickets(tezos_account, ticketer, 1_000_000)

Wrapping FA2 token, address: `KT1Wsem3Uka7wx5cGnk4nRZfyWVX5J7aWVX4`, id: `9000` tokens to tickets:
  - Token holder: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Ticketer: `KT1Cm6eSN3aEPPuTEQEXkehYBTu6caG5ZNxp`
  - Amount: `1_000_000`
Successfully wrapped, tx hash: `opYEuLQpwtuMrxHYwoCUkuKRWcuH5go5ESRyJX8psBCnFPjeW5A`
